# Derivation: Linear Discriminant Analysis (LDA) via SVD
How should we project high-dimensional labeled data onto a lower-dimensional space to best separate the classes? Linear Discriminant Analysis (LDA) answers this by finding directions that maximize between-class variance (spreading class means apart) while minimizing within-class variance (keeping each class compact).

> **Learning Objectives:**
>
> By the end of this lesson, you should be able to:
> * Formulate the LDA objective using within-class and between-class scatter matrices
> * Derive the generalized eigenvalue problem that maximizes Fisher's criterion
> * Develop a numerically stable SVD-based algorithm that avoids explicit matrix inversion

Let's get started!
___

## Theory: Scatter matrices and the generalized eigenvalue problem

**Notation and setup.** We have $N$ labeled samples $\{(\mathbf{x}_i, y_i)\}_{i=1}^N$, where $\mathbf{x}_i\in\mathbb{R}^d$ and $y_i\in\{1,\dots,K\}$ is the class label. For each class $k$, let $n_k$ be the number of samples and define:
$$
\boldsymbol{\mu}_k = \frac{1}{n_k}\sum_{i\,\in\,\mathcal{C}_k}\mathbf{x}_i, \qquad \boldsymbol{\mu} = \frac{1}{N}\sum_{i=1}^N\mathbf{x}_i.
$$

**Scatter matrices.** The **within-class scatter** and **between-class scatter** measure two aspects of the data:
$$
\begin{align*}
\mathbf{S}_w &= \sum_{k=1}^K\sum_{i\,\in\,\mathcal{C}_k}(\mathbf{x}_i-\boldsymbol{\mu}_k)(\mathbf{x}_i-\boldsymbol{\mu}_k)^{\top},\\[4pt]
\mathbf{S}_b &= \sum_{k=1}^K n_k(\boldsymbol{\mu}_k-\boldsymbol{\mu})(\boldsymbol{\mu}_k-\boldsymbol{\mu})^{\top}.
\end{align*}
$$
$\mathbf{S}_w$ quantifies within-class covariance (how tight each class is), while $\mathbf{S}_b$ quantifies between-class separation (how spread apart class means are). Together: $\mathbf{S}_t = \mathbf{S}_w + \mathbf{S}_b$.

**Fisher's criterion.** To find a projection direction $\mathbf{w}$ that best separates classes, we look for one-dimensional projections $z = \mathbf{w}^{\top}\mathbf{x}$. Under projection, the scatter transforms to:
$$
\sigma_w^2(\mathbf{w}) = \mathbf{w}^{\top}\mathbf{S}_w\mathbf{w}, \qquad \sigma_b^2(\mathbf{w}) = \mathbf{w}^{\top}\mathbf{S}_b\mathbf{w}.
$$
Fisher's criterion maximizes their ratio:
$$
J(\mathbf{w}) = \frac{\mathbf{w}^{\top}\mathbf{S}_b\mathbf{w}}{\mathbf{w}^{\top}\mathbf{S}_w\mathbf{w}}.
$$

To find the optimal $\mathbf{w}$, impose the constraint $\mathbf{w}^{\top}\mathbf{S}_w\mathbf{w}=1$ (normalization with respect to within-class scatter) and use Lagrange multipliers:
$$
\mathcal{L}(\mathbf{w},\lambda)=\mathbf{w}^{\top}\mathbf{S}_b\mathbf{w}-\lambda(\mathbf{w}^{\top}\mathbf{S}_w\mathbf{w}-1).
$$
Setting $\nabla_\mathbf{w}\mathcal{L}=0$ yields
$$
\boxed{\mathbf{S}_b\mathbf{w} = \lambda\mathbf{S}_w\mathbf{w}},
$$
the **generalized eigenvalue problem**. The optimal $\mathbf{w}$ is the eigenvector for the largest eigenvalue $\lambda$, which also equals $\max J(\mathbf{w})$.

**Multiple discriminant directions.** For $K > 2$ classes, one direction is rarely sufficient. We extend the criterion by collecting $r$ projection vectors as $\mathbf{W}=[\mathbf{w}_1,\dots,\mathbf{w}_r]\in\mathbb{R}^{d\times r}$ and optimizing:
$$
J(\mathbf{W}) = \mathrm{tr}\left[\left(\mathbf{W}^{\top}\mathbf{S}_w\mathbf{W}\right)^{-1}\left(\mathbf{W}^{\top}\mathbf{S}_b\mathbf{W}\right)\right].
$$

The optimal columns still satisfy the same generalized eigenvalue equation, solved sequentially: $\mathbf{w}_1$ is the top eigenvector, $\mathbf{w}_2$ the next, and so on. Since $\mathrm{rank}(\mathbf{S}_b) \le K-1$, we can have at most $K-1$ meaningful discriminant directions.

___

## Computing LDA: The SVD approach

We've now formulated the problem: find eigenvectors of $\mathbf{S}_b\mathbf{w} = \lambda\mathbf{S}_w\mathbf{w}$. A naive approach is to invert: rewrite as $(\mathbf{S}_w^{-1}\mathbf{S}_b)\mathbf{w} = \lambda\mathbf{w}$ and compute eigenvectors. But this fails when $\mathbf{S}_w$ is singular (which happens whenever $d > N$ or when classes are low-rank). Direct inversion amplifies numerical errors and may not even be possible.

Instead, we use **SVD to construct a stable whitening transform**. This replaces the ill-conditioned inversion with a well-conditioned change of variables.

**Form the centered data matrix.** Collect all data points centered by their class mean:
$$
\mathbf{H} = [\mathbf{x}_1-\boldsymbol{\mu}_{y_1},\dots,\mathbf{x}_N-\boldsymbol{\mu}_{y_N}]\in\mathbb{R}^{d\times N}.
$$
Then $\mathbf{S}_w = \mathbf{H}\mathbf{H}^{\top}$ follows directly from the definition.

**Decompose via thin SVD.** Now compute
$$
\mathbf{H} = \mathbf{U}\mathbf{\Sigma}\mathbf{V}^{\top},\qquad \mathbf{U}\in\mathbb{R}^{d\times r_w},\quad \mathbf{\Sigma}\in\mathbb{R}^{r_w\times r_w},\quad r_w = \mathrm{rank}(\mathbf{H}).
$$
This gives us $\mathbf{S}_w = \mathbf{U}\mathbf{\Sigma}^2\mathbf{U}^{\top}$. The key insight: we have the spectral decomposition without needing to invert anything.

**Whiten via a change of variables.** Instead of working with $\mathbf{w}$ directly, define a new variable $\mathbf{a}$ that lives in the reduced $r_w$-dimensional space, and connect them via $\mathbf{w}=\mathbf{U}\mathbf{\Sigma}^{-1}\mathbf{a}$. Substituting this into the generalized eigenvalue problem $\mathbf{S}_b\mathbf{w} = \lambda\mathbf{S}_w\mathbf{w}$ gives:
$$
\mathbf{S}_b(\mathbf{U}\mathbf{\Sigma}^{-1}\mathbf{a}) = \lambda(\mathbf{U}\mathbf{\Sigma}^2\mathbf{U}^{\top})(\mathbf{U}\mathbf{\Sigma}^{-1}\mathbf{a}).
$$
Simplifying (the $\mathbf{U}^{\top}\mathbf{U}$ terms cancel, the $\mathbf{\Sigma}$ terms combine), we obtain a standard eigenvalue problem in the reduced space:
$$
\boxed{(\mathbf{\Sigma}^{-1}\mathbf{U}^{\top}\mathbf{S}_b\mathbf{U}\mathbf{\Sigma}^{-1})\mathbf{a} = \lambda\mathbf{a}}.
$$

**Recover the original directions.** Once we find the eigenvectors $\mathbf{a}$ of this reduced problem, we transform back to the original space:
$$
\mathbf{w} = \mathbf{U}\mathbf{\Sigma}^{-1}\mathbf{a}.
$$

> __Numerical note:__
>
> If $\mathbf{S}_w$ is singular, use the pseudoinverse $\mathbf{\Sigma}^{+}$ (keep only singular values above a tolerance). This makes the LDA solution well-defined and avoids explicit inversion.

___

## SVD-based LDA algorithm
__Initialization__: Given labeled data $\{(\mathbf{x}_i,y_i)\}_{i=1}^N$ with $K$ classes, compute class means $\boldsymbol{\mu}_k$, overall mean $\boldsymbol{\mu}$, and class counts $n_k$. Choose a nonnegative tolerance $\tau$ for truncating small singular values (e.g., $\tau\approx 0$ or a small multiple of machine precision).

- Build the within-class centered matrix $\mathbf{H}=[\mathbf{x}_i-\boldsymbol{\mu}_{y_i}]$.
- Compute the thin SVD: $\mathbf{H}=\mathbf{U}\mathbf{\Sigma}\mathbf{V}^{\top}$.
- Form the between-class mean matrix $\mathbf{M}=[\sqrt{n_1}(\boldsymbol{\mu}_1-\boldsymbol{\mu}),\dots,\sqrt{n_K}(\boldsymbol{\mu}_K-\boldsymbol{\mu})]$ so that $\mathbf{S}_b=\mathbf{M}\mathbf{M}^{\top}$.
- Whiten with SVD: $\mathbf{B}=\mathbf{\Sigma}^{-1}\mathbf{U}^{\top}\mathbf{M}$ (use $\mathbf{\Sigma}^{+}$ and discard singular values $<\tau$).
- Solve the reduced eigenproblem: $\mathbf{B}\mathbf{B}^{\top}\mathbf{a}=\lambda\mathbf{a}$ and order eigenpairs by descending $\lambda$.
- Recover discriminant directions: $\mathbf{w}=\mathbf{U}\mathbf{\Sigma}^{-1}\mathbf{a}$ and keep the top $r\le K-1$ vectors.

__Output__: Return the LDA projection matrix $\mathbf{W}_r=[\mathbf{w}_1,\dots,\mathbf{w}_r]$ and the associated eigenvalues $\lambda_1\ge\cdots\ge\lambda_r$.

___

## Summary
LDA reduces to a generalized eigenvalue problem that can be solved stably with SVD.

> __Key Takeaways:__
>
> * LDA maximizes Fisher's criterion—the ratio of between-class to within-class scatter—yielding the generalized eigenvalue problem $\mathbf{S}_b\mathbf{w}=\lambda\mathbf{S}_w\mathbf{w}$, with at most $K-1$ nonzero discriminant directions.
> * The thin SVD of the within-class centered data matrix $\mathbf{H}$ provides $\mathbf{S}_w = \mathbf{U}\mathbf{\Sigma}^2\mathbf{U}^{\top}$, enabling a numerically stable solution that naturally handles singular scatter matrices without explicit inversion.
> * Whitening via $\mathbf{Z}=\mathbf{\Sigma}^{-1}\mathbf{U}^{\top}$ transforms the generalized eigenproblem into a standard eigenproblem in reduced space: $\left(\mathbf{\Sigma}^{-1}\mathbf{U}^{\top}\mathbf{S}_b\mathbf{U}\mathbf{\Sigma}^{-1}\right)\mathbf{a} = \lambda\mathbf{a}$.

___